In [79]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import os
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn import metrics
from scipy import stats
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.svm import SVC

## Loading the Training and Testing  data

In [80]:
#display the first 5 rows of the training data 
df = pd.read_csv('train.csv')
df.head()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,id
0,0,3,2,14,18,2,2.0,14,12,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2583
1,1,14,1,3,10,2,NaN,8,8,63.0,...,8.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,6830
2,1,14,1,13,10,8,8.0,10,10,331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4840
3,1,38,2,9,20,18,13.0,6,7,200.0,...,9.0,8.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,5508
4,1,24,2,14,20,6,6.0,20,17,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4828


In [81]:
#display the first 5 rows of the test data 
df1 = pd.read_csv('test.csv')
df1.head()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,id
0,0,5,2,2,16,3,NaN,13,13,52.0,...,7.0,8.0,6.0,8.0,NaN,NaN,NaN,NaN,NaN,934
1,0,33,2,14,18,6,6.0,4,8,368.0,...,8.0,7.0,7.0,8.0,6.0,7.0,6.0,5.0,5.0,6539
2,1,6,2,9,20,10,16.0,15,19,212.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6757
3,1,26,2,2,19,15,NaN,8,10,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2275
4,0,29,2,7,16,7,7.0,10,5,162.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1052


In [82]:
#dimension of the data 
df.shape, df1.shape

((5909, 192), (2469, 191))

In [83]:
# sum the dublicates 
df.duplicated().sum()

0

In [84]:
# display the feature
[x for x in df.columns]

['gender',
 'idg',
 'condtn',
 'wave',
 'round',
 'position',
 'positin1',
 'order',
 'partner',
 'pid',
 'match',
 'int_corr',
 'samerace',
 'age_o',
 'race_o',
 'pf_o_att',
 'pf_o_sin',
 'pf_o_int',
 'pf_o_fun',
 'pf_o_amb',
 'pf_o_sha',
 'attr_o',
 'sinc_o',
 'intel_o',
 'fun_o',
 'amb_o',
 'shar_o',
 'like_o',
 'prob_o',
 'met_o',
 'age',
 'field',
 'field_cd',
 'undergra',
 'mn_sat',
 'tuition',
 'race',
 'imprace',
 'imprelig',
 'from',
 'zipcode',
 'income',
 'goal',
 'date',
 'go_out',
 'career',
 'career_c',
 'sports',
 'tvsports',
 'exercise',
 'dining',
 'museums',
 'art',
 'hiking',
 'gaming',
 'clubbing',
 'reading',
 'tv',
 'theater',
 'movies',
 'concerts',
 'music',
 'shopping',
 'yoga',
 'exphappy',
 'expnum',
 'attr1_1',
 'sinc1_1',
 'intel1_1',
 'fun1_1',
 'amb1_1',
 'shar1_1',
 'attr4_1',
 'sinc4_1',
 'intel4_1',
 'fun4_1',
 'amb4_1',
 'shar4_1',
 'attr2_1',
 'sinc2_1',
 'intel2_1',
 'fun2_1',
 'amb2_1',
 'shar2_1',
 'attr3_1',
 'sinc3_1',
 'fun3_1',
 'intel3_1',
 '

In [85]:
df.corr()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,id
gender,1.000000,0.032132,-0.000875,-0.004192,0.017755,-0.004047,0.000410,0.009850,0.010318,-0.056275,...,-0.169387,0.011476,-0.153701,-0.066626,-0.133302,-0.277085,0.080227,-0.065562,0.069091,0.006340
idg,0.032132,1.000000,0.330587,0.093823,0.391918,0.164705,0.174651,0.161976,0.139034,0.088372,...,-0.050350,-0.060674,-0.041080,-0.005502,0.001764,-0.018915,-0.093206,-0.061079,-0.145645,-0.036693
condtn,-0.000875,0.330587,1.000000,0.219735,0.820898,0.331013,0.306722,0.331402,0.322467,0.218009,...,0.044158,0.061796,0.069298,0.027447,0.077571,-0.087502,0.051449,0.106486,0.123314,-0.044675
wave,-0.004192,0.093823,0.219735,1.000000,0.228917,0.079820,0.061166,0.093478,0.087904,0.996714,...,0.036948,-0.104562,0.053971,0.000520,0.092556,-0.001751,0.014611,0.088716,0.047841,-0.001863
round,0.017755,0.391918,0.820898,0.228917,1.000000,0.380632,0.368352,0.397952,0.390320,0.218901,...,0.036600,0.022696,0.059530,0.013107,0.034647,-0.035242,-0.012896,0.104829,0.092951,-0.029700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sinc5_3,-0.277085,-0.018915,-0.087502,-0.001751,-0.035242,0.111991,0.125594,-0.035897,-0.002601,0.021372,...,0.608213,0.315857,0.180240,0.122856,0.190198,1.000000,0.498313,0.335823,0.261307,0.051102
intel5_3,0.080227,-0.093206,0.051449,0.014611,-0.012896,0.089283,0.106791,-0.004632,0.003345,0.009585,...,0.288062,0.683871,0.240322,0.268646,0.355876,0.498313,1.000000,0.262987,0.422743,0.003512
fun5_3,-0.065562,-0.061079,0.106486,0.088716,0.104829,0.135484,0.232412,0.016419,0.031795,0.096788,...,0.160204,0.295997,0.771447,0.331533,0.412403,0.335823,0.262987,1.000000,0.379768,0.003349
amb5_3,0.069091,-0.145645,0.123314,0.047841,0.092951,0.058825,0.030543,0.029174,0.050511,0.043115,...,0.135499,0.353093,0.429332,0.619087,0.194078,0.261307,0.422743,0.379768,1.000000,-0.007695


In [86]:
#info about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5909 entries, 0 to 5908
Columns: 192 entries, gender to id
dtypes: float64(173), int64(11), object(8)
memory usage: 8.7+ MB


In [87]:
#sum the nulls and sort them descendingly
df.isnull().sum().sort_values(ascending = False)

num_in_3    5449
numdat_3    4849
expnum      4627
amb7_2      4519
sinc7_2     4519
            ... 
position       0
round          0
wave           0
condtn         0
id             0
Length: 192, dtype: int64

In [88]:
# retreive the columns that has null values > 65% and since more than .65 has null values 
null_cols = df.columns[df.isnull().sum()/df.shape[0] >.65]
null_cols

Index(['expnum', 'attr7_2', 'sinc7_2', 'intel7_2', 'fun7_2', 'amb7_2',
       'shar7_2', 'numdat_3', 'num_in_3', 'attr7_3', 'sinc7_3', 'intel7_3',
       'fun7_3', 'amb7_3', 'shar7_3', 'shar2_3', 'attr5_3', 'sinc5_3',
       'intel5_3', 'fun5_3', 'amb5_3'],
      dtype='object')

In [89]:
df.drop(columns=null_cols, inplace = True)
df1.drop(columns=null_cols, inplace = True)

In [90]:
df.shape,df1.shape

((5909, 171), (2469, 170))

In [91]:
df.isna().sum().sort_values(ascending = False)

shar4_3     3838
fun2_3      3838
attr4_3     3838
sinc4_3     3838
intel4_3    3838
            ... 
position       0
round          0
wave           0
condtn         0
id             0
Length: 171, dtype: int64

In [92]:
#define the input and the output
X = df.drop('match',axis = 1)
y = df['match']

## splitting the data into 80% training and 20% Testing

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [94]:
object_col=X_train.select_dtypes(include=['object'])
for col in object_col.columns.values:
    df[col] = X_train[col].astype("category")

## A Tunable Pipeline

In fact, any preprocessing steps can be considered as part of model and different configurations for these steps can be tunable. We can combine preprocessing steps and model as a single tunable pipeline with hyper-parameters.

In [95]:
# we extract numeric features and categorical features names
# for later use

# numeric features can be selected by: (based on the df2.info() output )
features_numeric = list(X_train.select_dtypes(include=['float64', 'int64']))

# categorical features can be selected by: (based on the df2.info() output )
features_categorical = list(X_train.select_dtypes(include=['category']))

print('numeric features:', features_numeric)
print('categorical features:', features_categorical)

numeric features: ['gender', 'idg', 'condtn', 'wave', 'round', 'position', 'positin1', 'order', 'partner', 'pid', 'int_corr', 'samerace', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'age', 'field_cd', 'race', 'imprace', 'imprelig', 'goal', 'date', 'go_out', 'career_c', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1', 'fun3_1', 'intel3_1', 'amb3_1', 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1', 'attr', 'sinc', 'intel', 'fun', 'amb', 'shar', 'like', 'prob', 'met', 'match_es', 'attr1_s', 'sinc1_s', 'in

In [96]:
#importing important libraries
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0) #every time you call the numpy's other random function, the result will be the same

# define a pipe line for numeric feature preprocessing
# we gave them a name so we can set their hyperparameters
transformer_numeric = Pipeline(
    steps=[
        ('imputer', SimpleImputer()),#Imputation transformer for completing missing values.
        ('scaler', StandardScaler())]#Standardize features

)

# define a pipe line for categorical feature preprocessing
# we gave them a name so we can set their hyperparameters
transformer_categorical = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant')), #here I set the strategy for completting missing values  = constant
        ('onehot', OneHotEncoder(handle_unknown='ignore')) #Encode categorical features as a one-hot numeric
        #When handle_unknown parameter is set to ‘ignore’ and an unknown category is encountered during transform,
        #the resulting one-hot encoded columns for this feature will be all zeros.

    ]
)
# define the preprocessor 
# we gave them a name so we can set their hyperparameters
# we also specify what are the categorical 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_numeric, features_numeric),
        ('cat', transformer_categorical, features_categorical)
    ]
) 

## Grid Search with Cross-validation

We can tune the hyperparameters (including different preprocessing configurations using cross-validation and grid-search). Grid search tests all possible combination of hyperparameters by the options and ranges you specified.

In [97]:
# combine the preprocessor with the model as a full tunable pipeline
# we gave them a name so we can set their hyperparameters
RF_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('my_classifier', 
           RandomForestClassifier(), #I want to use random forest classifier
        )
    ]
)
RF_pipline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['gender', 'idg', 'condtn',
                                                   'wave', 'round', 'position',
                                                   'positin1', 'order',
                                                   'partner', 'pid', 'int_corr',
                                                   'samerace', 'age_o',
                                                   'race_o', 'pf_o_att',
                                                   'pf_o_sin', 'pf_o_int',
                                                   

In [126]:
# The pipeline object can be used like any sk-learn model
RF_pipline = full_pipline.fit(X_train, y_train)
RF_pipline.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

## 1st Trial Random Forest

In [99]:
# Grid Search with Cross-validation
# here we specify the search space
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean'], #I set it to mean
    'my_classifier__n_estimators': [50, 100, 150],  # ranges of n_estimators which are the number of trees to be used in the forest.
    # I set my  n_estimators ranges to [50, 100, 150]
    'my_classifier__max_depth':[30, 60, 90]  
    # I set my  max depth ranges to [30, 60, 90] which are The number of splits that each decision tree is allowed to make.
}

# five-fold cross-validation
# n_jobs means cucurrent number of jobs is 2
RF_GSearch = GridSearchCV(
    RF_pipline, param_grid, cv=5, verbose=1 # showing more 'wordy' information
    , n_jobs=2, 
    scoring='roc_auc')#The degree of separability/distinction or intermingling/crossover between the forecasts of the two classes is shown by the ROC-AUC.

RF_GSearch.fit(X, y) #fitting X and y

print('best score {}'.format(grid_search.best_score_)) #printing best score
print('best parameters {}'.format(grid_search.best_params_))#printing best parameters

Fitting 5 folds for each of 9 candidates, totalling 45 fits
best score 0.8535789105295903
best parameters {'my_classifier__max_depth': 170, 'my_classifier__n_estimators': 180, 'preprocessor__num__imputer__strategy': 'most_frequent'}


In [100]:
RF_GSearch.predict(df1)

array([0, 1, 0, ..., 0, 0, 0])

In [101]:
submission = pd.DataFrame() # creating dataframe for the submission
submission['id'] = df1['id'] #using the column index of x test values to fill submission['id'] column
submission['match'] = grid_search.predict_proba(df1)[:,1] #predicting the probabilities and filling submission['match'] column with their values
submission.to_csv('sample_submission_walkthrough.csv', index=False)#generating the submission file

## Trial one using RandomForest with Grid Search with Cross-validation
Cross-validation is a method for robustly estimating test-set performance (generalization) of a model. Grid-searching is the process of scanning the data to configure optimal parameters for a given model.

## Expectation
I expect to get the optimal hyperparamters that will give me the best performance and highest accuracy using random forest model with grid search and cross validation

## Observation
After running the code,the best hyperparameters that was defined by the grid search using cross validation were :

max_depth: 90

n_estimators: 150

with using imputer__strategy of: 'mean'

The model gave me a best score of 0.8597120781837011

## Plan
I'm going to set different ranges for the hyperparameters in the grid search with cross validation

n_estimators': [90, 180, 270 ]

max_depth':[30, 100, 170]

and I changed my imputer__strategy' to 'most_frequent'

and I changed number of folds to cv=10

## 2nd Trial Random Forest

In [102]:
# Grid Search with Cross-validation
# here we specify the search space
param_grid = {
    'preprocessor__num__imputer__strategy': ['most_frequent'], #I set it to most frequent
    'my_classifier__n_estimators': [90, 180, 270],  # ranges of n_estimators which are the number of trees to be used in the forest.
    # I set my  n_estimators ranges to [90, 180, 270]
    'my_classifier__max_depth':[30, 100, 170]  
    # I set my  max depth ranges to [30, 100, 170] which are The number of splits that each decision tree is allowed to make.
}

# four-fold cross-validation
# n_jobs means cucurrent number of jobs is -1
RF_GSearch = GridSearchCV(
    RF_pipline, param_grid, cv=10, verbose=1 # showing more 'wordy' information
    , n_jobs=-1, 
    scoring='roc_auc')#The degree of separability/distinction or intermingling/crossover between the forecasts of the two classes is shown by the ROC-AUC.

RF_GSearch.fit(X_train, y_train) #fitting X and y

print('best score {}'.format(grid_search.best_score_)) #printing best score
print('best parameters {}'.format(grid_search.best_params_))#printing best parameters

Fitting 10 folds for each of 9 candidates, totalling 90 fits
best score 0.8535789105295903
best parameters {'my_classifier__max_depth': 170, 'my_classifier__n_estimators': 180, 'preprocessor__num__imputer__strategy': 'most_frequent'}


In [55]:
RF_GSearch.predict(df1)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
submission = pd.DataFrame() # creating dataframe for the submission
submission['id'] = df1['id'] #using the column index of x test values to fill submission['id'] column
submission['match'] = grid_search.predict_proba(df1)[:,1] #predicting the probabilities and filling submission['match'] column with their values
submission.to_csv('RF_GS.csv', index=False)#generating the submission file

## 2nd Trial

## Expectation
Expected higher score after changing the hyperparameters 

## Observation
After running the code,the best hyperparameters that was defined by the grid search using cross validation were :

max_depth: 30

n_estimators: 2700

with using imputer__strategy of: 'most_frequent'

The model gave me a best score of 0.861641763794065

## 1st Trial Decision Tree

In [29]:
DT_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('my_classifier', 
           DecisionTreeClassifier(), #I want to use random forest classifier
        )
    ]
)
DT_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['gender', 'idg', 'condtn',
                                                   'wave', 'round', 'position',
                                                   'positin1', 'order',
                                                   'partner', 'pid', 'int_corr',
                                                   'samerace', 'age_o',
                                                   'race_o', 'pf_o_att',
                                                   'pf_o_sin', 'pf_o_int',
                                                   

In [30]:
DT_pipeline = DT_pipeline.fit(X_train,y_train)
DT_pipeline.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [31]:
decision_search={
    "my_classifierdes__criterion" : ["gini" , "entropy"],
     "my_classifierdes__max_depth" : [2 , 4 , 6 , 8, 10, 12],
     "my_classifierdes__min_samples_leaf": [10,11,20,30]
 }
decision_search = GridSearchCV(
    DT_pipeline, decision_search, cv=10, verbose=1, n_jobs=2, 
    scoring='roc_auc')

decision_search.fit(X_train, y_train)
print('best score {}'.format(decision_search.best_score_))
print('best score {}'.format(decision_search.best_params_))

Fitting 10 folds for each of 48 candidates, totalling 480 fits
best score 0.8154633972147636
best score {'my_classifierdes__criterion': 'entropy', 'my_classifierdes__max_depth': 4, 'my_classifierdes__min_samples_leaf': 30}


In [32]:
decision_search.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## Decision Tree 2nd Trial

In [33]:
decision_search={
    "my_classifierdes__criterion" : ["gini" , "entropy"],
     "my_classifierdes__max_depth" : [2, 4, 6, 5, 10, 15, 20, 25],
     "my_classifierdes__min_samples_leaf": [5,10,15,20,30],
 }
decision_search = GridSearchCV(
    decision_pipline, decision_search, cv=15, verbose=1, n_jobs=5,
    scoring='roc_auc')

decision_search.fit(X_train, y_train)
print('best score {}'.format(decision_search.best_score_))
print('best score {}'.format(decision_search.best_params_))

Fitting 15 folds for each of 80 candidates, totalling 1200 fits
best score 0.8140942286621417
best score {'my_classifierdes__criterion': 'entropy', 'my_classifierdes__max_depth': 5, 'my_classifierdes__min_samples_leaf': 30}


In [ ]:
submission = pd.DataFrame() # creating dataframe for the submission
submission['id'] = df1['id'] #using the column index of x test values to fill submission['id'] column
submission['match'] = decision_search.predict_proba(df1)[:,1] #predicting the probabilities and filling submission['match'] column with their values
submission.to_csv('DT_GS.csv', index=False)#generating the submission file

## Bayesian Search

## XGB 1st Trial 

In [57]:
XGBpipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('my_classifier', 
           XGBClassifier(),
        )
        
    ]
)
XGBbayes_search = BayesSearchCV(
    XGBpipline,
    {
    'my_classifier__max_depth':[4,5,7,],
    'my_classifier__n_estimators': [20, 50,100, 400,500,600],
    'my_classifier__learning_rate': [0.1, 0.5, 0.05]}
,
    # number of trials 
    n_iter=10,
    random_state=0,
    verbose=1,
    # we still use 
    cv=10, scoring='roc_auc'
)

XGBbayes_search.fit(X_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
best score 0.8773251599351495
best score OrderedDict([('my_classifier__learning_rate', 0.05), ('my_classifier__max_depth', 7), ('my_classifier__n_estimators', 500)])


In [59]:
XGBbayes_search.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [60]:
submission = pd.DataFrame() # creating dataframe for the submission
submission['id'] = df1['id'] #using the column index of x test values to fill submission['id'] column
submission['match'] = bayes_search.predict_proba(df1)[:,1] #predicting the probabilities and filling submission['match'] column with their values
submission.to_csv('XGB_BS.csv', index=False)#generating the submission file

## SVM 1st Trial

In [41]:
SVC_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('my_svc', SVC(class_weight='balanced',probability=True)) #setting probability to true to be able to use probna() later when submission
        #class_weight='balanced' assigns the class weights inversely proportional to their respective frequencies
    ]
)


# define ranges for bayes search
bayes_search = BayesSearchCV(
    SVC_pipline,
    {
        'my_svc__C': Real(1e-8, 1e+8, prior='log-uniform'), #ranges for c which is the Penalty parameter of the error term.
        'my_svc__gamma': Real(1e-9, 1e+1, prior='log-uniform'), #ranges for gamma, the gamma parameter defines how far the influence of a single training example reaches
        'my_svc__degree': Integer(1,8), #degree ranges,the degree of the polynomial kernel function 
        'my_svc__kernel': Categorical(['poly','linear', 'rbf']), #kernel ranges,A kernel is a function used in SVM for helping to solve problems
     
    },
    # number of trials 
    n_iter=6,
    random_state=0,#the random_state parameter is used for initializing the internal random number generator
    verbose=1,# showing more 'wordy' information
    cv=5,
)

bayes_search.fit(X_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best parameters {}'.format(bayes_search.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
best score 0.8254702059353223
best parameters OrderedDict([('my_svc__C', 0.00013612772657396573), ('my_svc__degree', 8), ('my_svc__gamma', 1.214461458157585), ('my_svc__kernel', 'poly')])


In [105]:
bayes_search.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [106]:
submission = pd.DataFrame() # creating dataframe for the submission
submission['id'] = df1['id'] #using the column index of x test values to fill submission['id'] column
submission['match'] = bayes_search.predict_proba(df1)[:,1] #predicting the probabilities and filling submission['match'] column with their values
submission.to_csv('SVC_BS.csv', index=False)#generating the submission file

## Random_search

## 1st Trial Decision Tree

In [128]:
decision_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('my_classifierdes', 
        DecisionTreeClassifier(),
        )
        
    ]
)
decision_search={
    "my_classifierdes__criterion" : ["gini" , "entropy"],
     "my_classifierdes__max_depth" : [2 , 4 , 6 , 8, 10, 12],
     "my_classifierdes__min_samples_leaf": [10,12,15,20,25,30]
 }
decision_search = RandomizedSearchCV(
    decision_pipline, decision_search, cv=10, verbose=1, n_jobs=2, n_iter=10,
    scoring='roc_auc')

decision_search.fit(X_train, y_train)
print('best score {}'.format(decision_search.best_score_))
print('best score {}'.format(decision_search.best_params_))


Fitting 10 folds for each of 10 candidates, totalling 100 fits
best score 0.8134462813909783
best score {'my_classifierdes__min_samples_leaf': 10, 'my_classifierdes__max_depth': 4, 'my_classifierdes__criterion': 'entropy'}


## 1st Trial Decision Tree

In [148]:
decision_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('my_classifierdes', 
        DecisionTreeClassifier(),
        )
        
    ]
)
decision_search={
    "my_classifierdes__criterion" : ["gini" , "entropy"],
     "my_classifierdes__max_depth" : [2 , 4 , 6 , 8, 10, 12],
     "my_classifierdes__min_samples_leaf": [10,12,15,22,25,30]
 }
decision_search = RandomizedSearchCV(
    decision_pipline, decision_search, cv=10, verbose=1, n_jobs=1, n_iter=10,
    scoring='roc_auc')

decision_search.fit(X_train, y_train)
print('best score {}'.format(decision_search.best_score_))
print('best score {}'.format(decision_search.best_params_))

Fitting 10 folds for each of 10 candidates, totalling 100 fits
best score 0.8158823078345669
best score {'my_classifierdes__min_samples_leaf': 25, 'my_classifierdes__max_depth': 4, 'my_classifierdes__criterion': 'entropy'}


## XGB 2nd Trial 

In [121]:
random_search = RandomizedSearchCV(
    XGBpipline, param_grid, cv=5, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')

random_search.fit(X_train, y_train)

print('best score {}'.format(random_search.best_score_))
print('best score {}'.format(random_search.best_params_))

C:\Users\Ahmed Mahmoud\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 9 candidates, totalling 45 fits
best score 0.8750893640716185
best score {'preprocessor__num__imputer__strategy': 'most_frequent', 'my_classifier__n_estimators': 180, 'my_classifier__max_depth': 30}


In [123]:
submission = pd.DataFrame() # creating dataframe for the submission
submission['id'] = df1['id'] #using the column index of x test values to fill submission['id'] column
submission['match'] = random_search.predict_proba(df1)[:,1] #predicting the probabilities and filling submission['match'] column with their values
submission.to_csv('XGB_RS.csv', index=False)#generating the submission file

## Our input
A data set containing lots of information about each participent in a speed dating event,such as his unique subject number,wether he was a male or female, information about each wave such as the number of people he/she met in a wave,station number where met partner,partner's id, wether the participant and the partner were the same race, information about the partner him/her self such as their race,their age,their decision and their rating from 1 to 6...etc.

The dataset also contained information collected from a Survey filled out by students that are interested in participating in order to register for the event Like age,field pf study and each field has its own code in our data set,their Median SAT score for their undergraduate institution and also some questions like How important was it to him/her (on a scale of 1-10) that a person they date be of the same racial/ethnic background or e be of the same religious background, and where were they originally from amd the zip code for the place they grew up in, their goal of paticipating in the event , their career, how happy does he/she expect to be with the people they meet during the speed-dating event .. etc.

It also had informatiom out of a survey Filled out by subjects after each date during the event, the survey required information like their id,their decision and answers for questions about each person they meet like their attributes,how much they like the person..etc.

It has also answers from each participent on Half way through meeting asking about what We want to know what each participent look for in the opposite sex.

Also it has information collected from a Survey filled out the day after participating in the event like how satisfied were they with the people they met,What do they think the opposite sex looks for in a date,How does each participent think he/she measures up...etc.

Last but not least,Subjects filled out 3-4 weeks after they had been sent their matches and the data set included information regarding that such as How many of the matches each paticipent had has he/she contacted to set up a date, Have the participent been on a date with any of his/her matches and how many....etc.

## Our output
Our model is required to predict the outcome of a specific speed dating session based on the profile of two people in order to implement a recommendation system to better match people in speed dating events. we are going to predict the probability that the dating session will lead to a successful match.

## What data mining function is required?
binary classification using pipelines,grid search,Random Search and Bayesian Search.

## What could be the challenges?
Developing a successful solution to our problem , complex data,datasets can include complex data elements ,another thing is that we have to make sure that our algorithm must be efficient and scalable to extract information from the big data and we should have enough knowledge and experience in order to use them if we needed to improve our algorithms

## What is the impact?
Our model predictions is going to implement a recommendation system to better match people in speed dating events which will make it easier when it comes to dating

## What is an ideal solution?
An ideal solution in my opinion will be measured in terms of metrics and performances

## What is the experimental protocol used and how was it carried out? What preprocessing steps are used?
when it comes to the preprocessing steps I made a pipeline performing the following on the training set and testing set :

SimpleImputer(),Imputation transformer for completing missing values.

one hot encoder to transform categorical data to numerical data

stadnard scalar to standard scale my data

The experimental protocol I used in my code is cross validation

## Why a simple linear regression model (without any activation function) is not good for classification task, compared to Perceptron/Logistic regression?
A simple linear regression model without any activation function is not suitable for classification tasks because it does not provide a binary output, which is necessary for classification. In simple linear regression, the output is a continuous value that can take any value, including negative values and values greater than 1. This makes it difficult to interpret the output as a class label.

On the other hand, Perceptron/Logistic regression models are specifically designed for classification tasks. They use activation functions, such as the sigmoid function, to squash the output of the linear regression model between 0 and 1, which can be interpreted as a probability of belonging to a particular class. This makes it easier to convert the output of the model into a binary classification label.

Additionally, Perceptron/Logistic regression models are more robust to outliers and noisy data compared to simple linear regression models. This is because they use a threshold function to determine the class label, which is less affected by the values of individual data points.

Overall, Perceptron/Logistic regression models are better suited for classification tasks compared to simple linear regression models because they produce binary outputs, are more interpretable, and are more robust to outliers and noisy data.

## What's a decision tree and how it is different to a logistic regression model?
A decision tree is a type of machine learning algorithm used for both classification and regression tasks. It involves creating a tree-like model of decisions and their possible consequences. The tree is constructed by recursively splitting the data into subsets based on the values of the input features, until each subset contains only one class or a maximum depth is reached. The final result is a series of if-then statements that can be used to make predictions on new data.

Logistic regression, on the other hand, is a statistical model used for binary classification tasks. It involves modeling the probability of an event occurring based on one or more input variables. The goal of logistic regression is to find the coefficients of the input variables that maximize the likelihood of the observed data.

The main difference between decision trees and logistic regression models is in their approach to modeling the relationship between the input variables and the output variable. Decision trees recursively partition the input space into regions that correspond to specific class labels, whereas logistic regression models estimate the probability of belonging to a specific class based on the values of the input variables.

Decision trees are generally more interpretable than logistic regression models, as they provide a clear and intuitive representation of the decision-making process. They can also handle non-linear relationships between the input variables and the output variable, and are more robust to noise and outliers. However, they can be prone to overfitting, especially when the tree is deep or the data is high-dimensional.

Logistic regression models, on the other hand, are generally more efficient and have better statistical properties, such as convergence guarantees and asymptotic normality of the parameter estimates. They can also handle continuous and categorical input variables, and are less prone to overfitting than decision trees. However, they can be less interpretable than decision trees, especially when the model has many input variables or complex interactions between them

## What's the difference between grid search and random search?
Grid search and random search are two popular hyperparameter tuning techniques used in machine learning.

Grid search involves exhaustively searching over a pre-defined set of hyperparameters by evaluating the model performance on a validation set for each combination of hyperparameters. Grid search is systematic and guarantees that all possible combinations of hyperparameters are evaluated, but it can be computationally expensive and time-consuming, especially for a large number of hyperparameters or large ranges of hyperparameter values.

Random search, on the other hand, involves randomly sampling from a pre-defined distribution of hyperparameters and evaluating the model performance on a validation set for each set of hyperparameters. Random search is less systematic than grid search, but can be more efficient in exploring the hyperparameter space, especially when the number of hyperparameters is large or the optimal hyperparameters are not known in advance. Random search can also help avoid local minima and discover unexpected combinations of hyperparameters.

Overall, grid search is a good choice when the hyperparameter space is relatively small and the optimal hyperparameters are expected to be near the center of the hyperparameter space, while random search is a good choice when the hyperparameter space is large or the optimal hyperparameters are not known in advance. In practice, a combination of both techniques can be used to achieve a good balance between systematic exploration and efficient search of the hyperparameter space.

## What's the difference between bayesian search and random search?

Bayesian search and random search are two popular hyperparameter tuning techniques used in machine learning.

Random search involves randomly sampling hyperparameters from a pre-defined distribution and evaluating the model performance on a validation set for each set of hyperparameters. Random search is flexible and easy to implement, but can be inefficient in high-dimensional search spaces and often requires a large number of samples to find optimal hyperparameters.

Bayesian search, on the other hand, uses a probabilistic model to predict the performance of different hyperparameter configurations and selects the next set of hyperparameters to evaluate based on the expected improvement over the current best configuration. Bayesian search is more efficient than random search in high-dimensional search spaces and can often find optimal hyperparameters with fewer samples.

The main difference between Bayesian search and random search is that Bayesian search uses a probabilistic model to guide the search process, while random search selects hyperparameters randomly. Bayesian search also takes into account the uncertainty in the model predictions and adjusts the search process accordingly, whereas random search does not consider the performance of previously evaluated hyperparameter configurations.

Overall, Bayesian search is a good choice when the hyperparameter space is high-dimensional and the optimal hyperparameters are not known in advance, while random search is a good choice when the hyperparameter space is small or the optimal hyperparameters are expected to be near the center of the search space. In practice, a combination of both techniques can be used to achieve a good balance between systematic exploration and efficient search of the hyperparameter space.